要运行此操作，请按“*Runtime*”，然后按“*运行*” ** free ** tesla Tesla T4 Google COLAB实例！
<div class =“ Align-Center”>
<a href =“ https://unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/mains/unsloth%20new%20new%20logo.png.png”
<a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/main/main/main/images/images/discord button.png button.png
<a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/mains/images/documentation%20green%20green%20breen%20button.png?png?raw=raw=true width width” <i>在<a href =“ https://github.com/unslothai/unsloth”> github </a> </i>⭐中
</div>

要在您自己的计算机上安装不塞，请按照我们的github页面上的安装说明[here](https://docs.unsloth.ai/get-started/installing-+-updating)上的安装说明。

您将学习如何做[data prep](#Data)，如何[train](#Train)，如何[run the model](#Inference)，＆xx_markDown_link_xx


### 消息

** New ** Unsploth现在支持培训OpenAi的新** GPT-oss **模型！您可以通过我们的** [Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)免费启动Finetune GPT-oss **！

Unsploth现在支持文本对语音（TTS）模型。阅读我们的xx_markDown_link_xx。

阅读我们的** [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning) **并查看我们的新** [Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune) ** Quants优先量优于其他量化方法！

请访问我们的所有文档，以获取我们的所有XX_MarkDown_link_xx和[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)。


### 安装

In [ ]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth
!pip install --upgrade transformers==4.53.2 "huggingface_hub>=0.34.0" "datasets>=3.4.1,<4.0.0"


### 不塞

In [ ]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # QWEN 14b 2x更快
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4位动态量子，以卓越的精度和低内存的使用
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [新]我们支持TTS模型！
] # https://huggingface.co/unsloth的更多型号

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Magistral-Small-2506-unsloth-bnb-4bit",
    max_seq_length = 2048,   # 上下文长度 - 可以更长，但使用更多内存
    load_in_4bit = True,     # 4bit使用更少的内存
    load_in_8bit = False,    # 更准确，使用2倍内存
    full_finetuning = False, # 我们现在有完整的填充！
    device_map = "balanced", # 使用2x Telsa T4S
    # token =“ hf _...”，＃如果使用门控模型，请使用一个
)

==((====))==  Unsloth 2025.6.1: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/164k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/201k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

现在，我们添加Lora适配器，因此我们只需要更新所有参数的1％至10％！

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # 选择任何数字> 0！建议8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # 最好选择alpha =等级*2
    lora_dropout = 0, # 支持任何人，但= 0已优化
    bias = "none",    # 支持任何人，但是=“无”
    # [NEW]“ Unsploth”使用的VRAM减少了30％，适合2倍批量的尺寸！
    use_gradient_checkpointing = "unsloth", # 在很长的背景下真实或“不整齐”
    random_state = 3407,
    use_rslora = False,   # 我们支持排名稳定的洛拉
    loftq_config = None,  # 和Loftq
)

Unsloth 2025.6.1 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name =“ data”> </a>
###数据准备
我们使用XX_Markdown_link_xx数据集，该数据集用于赢得XX_MarkDown_link_xx（AI数学奥林匹克运动会 - 进度奖2）挑战！我们采样了使用DeepSeek R1的10％可验证的推理轨迹，其精度> 95％。

In [6]:
from datasets import load_dataset
reasoning_dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

data/cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating cot split:   0%|          | 0/19252 [00:00<?, ? examples/s]

让我们看看数据集的结构：

In [7]:
reasoning_dataset

Dataset({
    features: ['expected_answer', 'problem_type', 'problem_source', 'generation_model', 'pass_rate_72b_tir', 'problem', 'generated_solution', 'inference_mode'],
    num_rows: 19252
})

现在，我们将推理数据集转换为对话格式：

In [8]:
def generate_conversation(examples):
    problems  = examples["problem"]
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [9]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

Map:   0%|          | 0/19252 [00:00<?, ? examples/s]

让我们看看第一个转换的行：

In [10]:
reasoning_conversations[0]

"<s>[SYSTEM_PROMPT]A user will ask you to solve a task. You should first draft your thinking process (inner monologue) until you have derived the final answer. Afterwards, write a self-contained summary of your thoughts (i.e. your summary should be succinct but contain all the critical steps you needed to reach the conclusion). You should use Markdown and Latex to format your response. Write both your thoughts and summary in the same language as the task posed by the user.\n\nYour thinking process must follow the template below:\n<think>\nYour thoughts or/and draft, like working through an exercise on scratch paper. Be as casual and as long as you want until you are confident to generate a correct answer.\n</think>\n\nHere, provide a concise summary that reflects your reasoning and presents a clear final answer to the user.\n\nProblem:[/SYSTEM_PROMPT][INST]Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.[/INST]<think>\nOkay, let's see. I ne

现在让我们看看数据集有多长时间：

In [11]:
print(len(reasoning_conversations))

19252


In [13]:
import pandas as pd
data = pd.concat([
    pd.Series(reasoning_conversations),
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

<a name =“ train”> </a>
###训练模型
现在，让我们使用huggingface trl的“ sfttrainer”！这里更多文档：[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)。我们执行60个步骤来加快速度，但是您可以将`num_train_epochs = 1`设置为完整运行，然后关闭`max_steps = none“。

In [17]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # 可以设置评估！
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2, # 使用GA来模仿批处理大小！
        warmup_steps = 5,
        # num_train_epochs = 1，＃将其设置为1个完整的训练运行。
        max_steps = 15,
        learning_rate = 2e-4, # 长达2e-5用于长期训练
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # 将其用于Wandb等
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/19252 [00:00<?, ? examples/s]

In [15]:
# @Title显示当前内存统计
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
14.594 GB of memory reserved.


让我们训练模型吧！要恢复训练跑

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,252 | Num Epochs = 1 | Total steps = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 184,811,520/12,893,967,360 (1.43% trained)


In [19]:
# @Title显示最终内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

417.0752 seconds used for training.
6.95 minutes used for training.
Peak reserved memory = 18.139 GB.
Peak reserved memory for training = 3.545 GB.
Peak reserved memory % of max memory = 81.851 %.
Peak reserved memory for training % of max memory = 15.997 %.


<a name =“推理”> </a>
###推理
让我们通过不舒服的本机推理来运行模型！根据“ Mistral”团队的说法，推理推理的推荐设置为`温度= 0.7，top_p = 0.95`

In [20]:
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^2 = 0."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # 必须添加一代
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # 增加输出的增加！
    temperature = 0.7, top_p = 0.95,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Okay, so I need to solve the equation (x + 2)^2 = 0. Hmm, let me think. The equation is saying that when I add x and 2 and then square the result, it equals zero. So, maybe I need to find x such that when I square something, it's zero. But squaring numbers usually gives a positive result unless the number itself is zero. So, if (x + 2)^2 = 0, then x + 2 must be zero, right? Because zero squared is zero. So, x + 2 = 0. Then, subtracting 2 from both sides, x = -2. That seems like the solution. Wait, but is that the only solution? Let me check. Since squaring a number can give the same result for a negative number as its positive counterpart, but in this case, the equation is set to zero, so the only number that squares to zero is zero itself. So, x + 2 must be zero. So, x = -2. That's it. So, the solution is x = -2. But just to be sure, maybe I should plug it back in. Let's see: if x = -2, then (x + 2)^2 = (-2 + 2)^2 = 0^2 = 0. Yes, that works. So, x = -2 is the solution. So, the

<a name =“ save”> </a>
###保存，加载固定模型
要将最终模型保存为Lora适配器，请使用HuggingFace的“ push_to_hub”进行在线保存或`save_pretaining'用于本地保存。

** [注意] **这仅保存洛拉适配器，而不是完整的模型。要节省16位或GGUF，请向下滚动！

In [21]:
model.save_pretrained("lora_model")  # 本地节省
tokenizer.save_pretrained("lora_model")
# model.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存
# tokenizer.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

现在，如果您想加载洛拉适配器，我们刚刚保存用于推理，请将`false``设置为true`：：true'：

In [22]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # 您用于培训的模型
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### 节省float16 vllm

我们还直接支持保存到`float16`。为float16选择`merged_16bit`或用于int4的merged_4bit`。我们还允许`Lora`适配器作为后备。使用`push_to_hub_merged`上传到您的拥抱脸部帐户！您可以访问https://huggingface.co/settings/tokens for个人令牌。

In [ ]:
# 合并为16位
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # 推向HF集线器
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 合并到4位
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # 推向HF集线器
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# 只是洛拉适配器
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False: # 推向HF集线器
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / LLAMA.CPP转换
要保存到`gguf' /`llama.cpp`，我们现在在本地支持它！我们克隆“ llama.cpp”，默认将其保存到Q8_0`。我们允许所有方法'q4_k_m`。使用`save_pretained_gguf`进行本地保存，然后`push_to_hub_gguf`将上传到HF。

一些支持的量化方法（我们的[Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)上的完整列表）：
*`Q8_0`-快速转换。大量资源使用，但通常可以接受。
*`q4_k_m`-推荐。使用q6_k作为注意到的一半。
*`q5_k_m`-推荐。使用q6_k进行一半的注意。wv和feed_forward.w2张量，else q5_k。

[** new **]要为Finetune和自动导出到Ollama，请尝试我们的[Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-olama.ipynb）

In [24]:
# 保存到8位Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# 请记住要访问https://huggingface.co/settings/tokens for代币！
# 并将HF更改为您的用户名！
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存至16位GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # 推向HF集线器
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存到Q4_K_M GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # 推向HF集线器
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 保存到多个GGUF选项 - 如果要多个，请更快！
if False:
    model.push_to_hub_gguf(
        "hf/model", # 将HF更改为您的用户名！
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # 在https://huggingface.co/settings/tokens上获取一个令牌
    )

现在，在llama.cpp中使用`model-unsloth.gguf`文件或`model-unsloth-q4_k_m.gguf`文件或基于UI的系统（例如JAN或OPEN webUI）。您可以安装JAN XX_MARKDOWN_LINK_XX并打开webui [here](https://github.com/janhq/jan)

我们完成了！如果您对不舒服有任何疑问，我们有一个XX_MarkDown_link_xx频道！如果您发现任何错误或想对最新的LLM内容进行更新，或者需要帮助，加入项目等，请随时加入我们的不和谐！

其他一些链接：
1。训练您自己的推理模型-Llama grpo笔记本XX_Markdown_link_xx-grpo.ipynb）
2。将芬太尼保存到奥尔马。 [here](https://github.com/open-webui/open-webui)-ollama.ipynb）
3。Llama3.2视觉燃烧 -  X射线照相用例。 [Discord](https://discord.gg/unsloth)-vision.ipynb）
6。请参见DPO，ORPO，继续预处理，对话finetuning等笔记本，以及我们的xx_markDown_link_xx上的更多内容！

<div class =“ Align-Center”>
  <a href =“ https://unsloth.ai”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/images/unsloth%20new%20new%20logo.png”
  <a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unsloth/unsloth/raw/main/main/main/images/images/discord.png，png
  <a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/images/images/images/documentation%20green%20green%20breen%20button.png?png?png?raw=true width =”

  如果您需要帮助，请加入DISCORD +⭐️<i>在<a href =“ https://github.com/unslothai/unsloth上”
</div>
